# Import packages

In [1]:
from src.utils import read_gtf, collapse_isoforms_to_proteoforms
import polars as pl

# Load datasets

In [2]:
final_classification = pl.read_parquet("nextflow_results/V47/final_classification.parquet")

In [3]:
orfanage_gtf = read_gtf("nextflow_results/V47/orfanage/orfanage.gtf")

In [4]:
annot_peptides_hybrid = read_gtf("nextflow_results/V47/orfanage/annot_peptides_hybrid.gtf", attributes=["gene_name", "transcript_id", "novelty", "detected"])

In [5]:
peptide_mapping = pl.read_parquet("nextflow_results/V47/orfanage/peptide_mapping.parquet")

# Start here

In [4]:
print(f"There are {final_classification.shape[0]} transcripts in the final classification")

There are 182371 transcripts in the final classification


In [5]:
print(f"There are {final_classification.filter(pl.col("structural_category").is_in(["novel_not_in_catalog", "novel_in_catalog"])).shape[0]} novel transcripts in the final classification")

There are 103953 novel transcripts in the final classification


In [7]:
print(f"There are {orfanage_gtf.filter(pl.col("feature")=="transcript").unique("transcript_id").shape[0]} transcripts that contain CDS predicted by ORFanage")

There are 162913 transcripts that contain CDS predicted by ORFanage


In [8]:
isoforms_to_proteoforms = collapse_isoforms_to_proteoforms(orfanage_gtf)

In [9]:
f"There are {isoforms_to_proteoforms.unique("base_isoform").shape[0]} unique ORFs."

'There are 81745 unique ORFs.'

In [8]:
protein_classification = pl.read_csv("nextflow_results/V47/orfanage/SFARI.protein_classification.tsv", separator="\t")

In [22]:
n_novel_protein_isoform = protein_classification\
    .join(
        isoforms_to_proteoforms.rename({"isoform": "pb"}),
        on = "pb",
        how = "left"
    ).unique("base_isoform")\
    .filter(pl.col("protein_classification_base").is_in(["pNIC", "pNNC"]))\
    .shape[0]
print(f"There are {n_novel_protein_isoform} novel protein isoforms")

There are 57489 novel protein isoforms


In [ ]:
n_novel_peptides = annot_peptides_hybrid\
    .filter(
        pl.col("detected")=="True",
        pl.col("novelty")=="novel"
    )\
    .unique("transcript_id")\
    .shape[0]

novel_peptides = annot_peptides_hybrid\
    .filter(
        pl.col("detected")=="True",
        pl.col("novelty")=="novel"
    )\
    .unique("transcript_id")\
    ["transcript_id"]

print(f"There are {n_novel_peptides} novel peptides that have been detected.")

There are 160 novel peptides that have been detected.


In [24]:
n_transcripts = peptide_mapping\
    .filter(
        pl.col("peptide").is_in(novel_peptides)
    ).unique("pb").shape[0]

novel_transcripts_validated = peptide_mapping\
    .filter(
        pl.col("peptide").is_in(novel_peptides)
    ).unique("pb")["pb"]

print(f"{n_transcripts} transcripts have been validated by these novel peptides.")

440 transcripts have been validated by these novel peptides.


In [29]:
n_genes = final_classification\
    .filter(
        pl.col("isoform").is_in(novel_transcripts_validated)
    ).unique("associated_gene").unique("associated_gene").shape[0]

print(f"There are {n_genes} that these transcripts map to.")


There are 151 that these transcripts map to.


# Novel genes

In [19]:
n_novel_genes = final_classification\
    .filter(
        pl.col("associated_gene").str.starts_with("novel") | pl.col("associated_gene").str.contains("_")
    ).unique("associated_gene").shape[0]

print(f"There are {n_novel_genes} novel genes.")

There are 2919 novel genes.


In [20]:
n_tx_novel_genes = final_classification\
    .filter(
        pl.col("associated_gene").str.starts_with("novel") | pl.col("associated_gene").str.contains("_")
    ).shape[0]

print(f"There are {n_tx_novel_genes} transcripts that map to novel genes.")

There are 5167 transcripts that map to novel genes.


In [23]:
final_classification\
    .filter(
        pl.col("associated_gene").str.starts_with("novel") | pl.col("associated_gene").str.contains("_"),
        pl.col("structural_category").is_in(["fusion"])
    )

isoform,chrom,strand,length,exons,structural_category,associated_gene,associated_transcript,ref_length,ref_exons,diff_to_TSS,diff_to_TTS,diff_to_gene_TSS,diff_to_gene_TTS,subcategory,RTS_stage,all_canonical,min_sample_cov,min_cov,min_cov_pos,sd_cov,FL,n_indels,n_indels_junc,bite,iso_exp,gene_exp,ratio_exp,FSM_class,coding,ORF_length,CDS_length,CDS_start,CDS_end,CDS_genomic_start,CDS_genomic_end,predicted_NMD,perc_A_downstream_TTS,seq_A_downstream_TTS,dist_to_CAGE_peak,within_CAGE_peak,dist_to_polyA_site,within_polyA_site,polyA_motif,polyA_dist,polyA_motif_found,ORF_seq,ratio_TSS,fl_assoc,cell_barcodes,containing_novel_spl
str,str,str,i32,i32,str,str,str,i32,i32,i32,i32,i32,i32,str,str,str,str,str,str,str,str,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,bool
"""PB.10.5""","""chr1""","""+""",3689,4,"""fusion""","""ENSG00000272438_ENSG0000029608…","""novel""",378,3,null,null,8,0,"""intron_retention""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""B""","""non_coding""",null,null,null,null,null,null,null,35.0,"""ATGAGGGGATGGATGAGAAG""",null,null,null,null,null,null,null,null,null,null,null,false
"""PB.10.6""","""chr1""","""+""",2589,2,"""fusion""","""ENSG00000272438_ENSG0000029608…","""novel""",657,2,null,null,22,0,"""intron_retention""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""B""","""non_coding""",null,null,null,null,null,null,null,30.0,"""GAGCTTGCCGCCCTAAAAAT""",null,null,null,null,null,null,null,null,null,null,null,false
"""PB.10.7""","""chr1""","""+""",5607,3,"""fusion""","""ENSG00000272438_ENSG0000029608…","""novel""",378,3,null,null,22,0,"""intron_retention""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""B""","""non_coding""",null,null,null,null,null,null,null,35.0,"""ATGAGGGGATGGATGAGAAG""",null,null,null,null,null,null,null,null,null,null,null,false
"""PB.10.12""","""chr1""","""+""",1609,4,"""fusion""","""ENSG00000272438_ENSG0000029608…","""novel""",378,3,null,null,38,0,"""multi-exon""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""B""","""non_coding""",null,null,null,null,null,null,null,30.0,"""TGAGGGGATGGATGAGAAGG""",null,null,null,null,null,null,null,null,null,null,null,false
"""PB.10.16""","""chr1""","""+""",3512,3,"""fusion""","""ENSG00000272438_ENSG0000029608…","""novel""",378,3,null,null,44,-4,"""intron_retention""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""B""","""non_coding""",null,null,null,null,null,null,null,40.0,"""ATGAGAAGGGCTCTAAGAGA""",null,null,null,null,null,null,null,null,null,null,null,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""PB.112376.19""","""chrX""","""-""",1594,6,"""fusion""","""CLIC2_ENSG00000224216""","""novel""",2623,6,null,null,-55,813,"""multi-exon""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""B""","""non_coding""",null,null,null,null,null,null,null,20.0,"""AAAAGGTCTTCTTGTTGTCC""",null,null,null,null,null,null,null,null,null,null,null,false
"""PB.112376.154""","""chrX""","""-""",1594,7,"""fusion""","""H2AB3_TMLHE""","""novel""",544,2,null,null,-19,-1,"""multi-exon""","""FALSE""","""canonical""",null,null,null,null,null,null,null,false,null,null,null,"""B""","""non_coding""",null,null,null,null,null,null,null,15.0,"""ATCTGCCCGCTTCAGTCACT""",null,null,null,null,null,null,null,null,null,null,null,false
"""PB.112382.1000""","""chrX""","""+""",2808,10,"""fusion""","""SPRY3_VAMP7""","""novel""",8931,3,null,null,0,-4,"""multi-exon""","""FALSE""","""canonical""",null,null,null,null,null,null,null,true,null,null,null,"""A""","""non_coding""",null,null,null,null,null,null,null,25.0,"""ATGATTTGCTTGTTTTAGAA""",null,null,null,null,null,null,null,null,null,null,null,false


In [24]:
final_classification["structural_category"].unique()

structural_category
str
"""novel_in_catalog"""
"""incomplete-splice_match"""
"""fusion"""
"""intergenic"""
"""full-splice_match"""
"""genic"""
"""antisense"""
"""moreJunctions"""
"""novel_not_in_catalog"""


In [26]:
final_classification.filter(pl.col("structural_category")=="intergenic").shape[0]

445

In [29]:
final_classification.filter(pl.col("structural_category")=="fusion").shape[0]

4332

In [31]:
df = final_classification.filter(pl.col("structural_category")=="fusion")